In [11]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

np.random.seed(10)
%matplotlib inline

# Set up dataset

In [12]:
gender = [i for i in range(2)]
location = [i for i in range(100)]
age = [i for i in range(5)]
device = [i for i in range(4)]
genre = [i for i in range(10)]
attributes = [gender,location,age,device,genre]
attrNames = ["gender","location","age","device","genre"]

In [13]:
attrComb = [[g,loc,a,d,ge] for g in gender for loc in location for a in age for d in device for ge in genre]

In [70]:
mu = 20000
sigma = 1000
numSupply = len(attrComb)
S = np.random.normal(mu, sigma, numSupply).round()

In [71]:
Supply = pd.DataFrame(attrComb,columns =["gender","location","age","device","genre"])

In [72]:
Supply['impression'] = S

In [73]:
Supply['name'] = Supply.index.values
Supply['name'] = Supply.name.apply(lambda x:'s'+str(x))

In [74]:
numDemand = 1000

D = np.random.normal(25,4,numDemand).round()*20000

attrTarget1 = np.random.normal(3,2,numDemand).round()
attrTarget1[attrTarget1<0] = 0
attrTarget1[attrTarget1>4] = 4

attrTarget2 = np.random.normal(3,2,numDemand).round()
attrTarget2[attrTarget2<0] = 1
attrTarget2[attrTarget2>4] = 3

targetVal = np.random.uniform(100)

ecpm = np.random.uniform(0,2,numDemand).round(3)
# V = np.random.uniform(0,1,numDemand).round(2)+0.01
V = np.random.uniform(1,4,numDemand).round()

In [75]:
Demand = pd.DataFrame([attrTarget1,attrTarget2],index=['target1','target2']).T
Demand = Demand.astype(int)
Demand['demand'] = D
Demand['penalty'] = ecpm + 0.005
Demand['penalty'] = 10


Demand['V'] = V

Demand['name'] = Demand.index.values
Demand['name'] = Demand.name.apply(lambda x:'d'+str(x))

In [76]:
Supply.impression.sum(),Demand.demand.sum()

(799564346.0, 499680000.0)

In [77]:
def pickChoice(x):
    length = len(attributes[x])
    v = np.random.uniform(length)
    return int(v)

def computeS(target1,target2,demand,v1,v2):
   
    attr1 = attrNames[int(target1)]
    attr2 = attrNames[int(target2)]
    if attr1 != attr2:
        impAvail = Supply[(Supply[attr1]==v1) & (Supply[attr2]==v2)].impression.sum()
    else:
        impAvail = Supply[(Supply[attr1]==v1) | (Supply[attr2]==v2)].impression.sum()
    return impAvail

In [78]:
Demand['v1'] = Demand.target1.apply(pickChoice)
Demand['v2'] = Demand.target2.apply(pickChoice)

Demand['S'] = Demand[['target1', 'target2', 'demand', 'v1', 'v2']].apply(lambda x:computeS(*x),axis=1)
Demand['theta'] = round(Demand['demand']/Demand['S'],4)
Demand = Demand.sort_values('theta',ascending=False)
Demand.loc[Demand['theta']>=1,'theta'] = 1



In [79]:
Demand.head()

,target1,target2,demand,penalty,V,name,v1,v2,S,theta
187,4,1,660000.0,10,2.0,d187,7,69,787742.0,0.8378
135,4,1,660000.0,10,2.0,d135,3,91,795219.0,0.8300
717,4,1,620000.0,10,2.0,d717,5,71,784486.0,0.7903
271,4,1,620000.0,10,2.0,d271,5,18,799621.0,0.7754
590,4,1,620000.0,10,3.0,d590,7,99,806589.0,0.7687


In [80]:
Supply.head()

,gender,location,age,device,genre,impression,name
0,0,0,0,0,0,22702.0,s0
1,0,0,0,0,1,20152.0,s1
2,0,0,0,0,2,19988.0,s2
3,0,0,0,0,3,21125.0,s3
4,0,0,0,0,4,20814.0,s4


# HWM

In [81]:
def findKsi(shat,s,d,step=101):
    
    if d>sum(shat):
        return np.inf
    
    probabilities = np.linspace(0,1,step)
    left,right = 0,step-1
    
    while (right > left):
        median = right - (right - left)//2
        p = probabilities[median]
        minimum = sum(np.minimum(shat,p*s))
        if minimum <= d:
            left = median + 1
        else:
            right = median-1
    
    return probabilities[median]
    

In [82]:
%%time
step = 101
SHat = Supply.copy()
Q = []
Delivered = []
for items in Demand[['target1', 'target2', 'demand', 'v1', 'v2']].values:
    
    target1,target2,dj,v1,v2 = items
    attr1 = attrNames[int(target1)]
    attr2 = attrNames[int(target2)]
  
    if attr1 != attr2:
        si = Supply[(Supply[attr1]==v1)&(Supply[attr2]==v2)].impression.values
        shati = SHat[(SHat[attr1]==v1)&(SHat[attr2]==v2)].impression.values

        ksi = findKsi(shati,si,dj,step)
    
        delivered = np.minimum(shati,si*ksi).round()

        SHat.loc[(SHat[attr1]==v1)&(SHat[attr2]==v2),"impression"] -= delivered
        Q.append(ksi)
        Delivered.append(sum(delivered))
        
    else:
        si = Supply[(Supply[attr1]==v1) | (Supply[attr2]==v2)].impression.values
        shati = SHat[(SHat[attr1]==v1) | (SHat[attr2]==v2)].impression.values

        ksi = findKsi(shati,si,dj,step)

        delivered = np.minimum(shati,si*ksi).round()

        SHat.loc[(SHat[attr1]==v1) | (SHat[attr2]==v2),"impression"] -= delivered
        Q.append(ksi)
        Delivered.append(sum(delivered))


CPU times: user 10.7 s, sys: 59.3 ms, total: 10.7 s
Wall time: 10.7 s


In [83]:
Demand['ksi'] = Q
Demand['delivered'] = Delivered

# Shale

In [84]:
Demand['alpha'] = np.array([0]*numDemand)
Supply['beta'] = np.array([0]*numSupply)

In [85]:
Supply.head()

,gender,location,age,device,genre,impression,name,beta
0,0,0,0,0,0,22702.0,s0,0
1,0,0,0,0,1,20152.0,s1,0
2,0,0,0,0,2,19988.0,s2,0
3,0,0,0,0,3,21125.0,s3,0
4,0,0,0,0,4,20814.0,s4,0


In [86]:
Demand.head()

,target1,target2,demand,penalty,V,name,v1,v2,S,theta,ksi,delivered,alpha
187,4,1,660000.0,10,2.0,d187,7,69,787742.0,0.8378,0.85,669582.0,0
135,4,1,660000.0,10,2.0,d135,3,91,795219.0,0.8300,0.82,652079.0,0
717,4,1,620000.0,10,2.0,d717,5,71,784486.0,0.7903,0.78,611899.0,0
271,4,1,620000.0,10,2.0,d271,5,18,799621.0,0.7754,0.78,623704.0,0
590,4,1,620000.0,10,3.0,d590,7,99,806589.0,0.7687,0.78,629139.0,0


In [87]:
def findDemandNeighbours(target1,target2,v1,v2):
    attr1 = attrNames[int(target1)]
    attr2 = attrNames[int(target2)]
    if attr1 != attr2:
        neighbors = Supply[(Supply[attr1]==v1) & (Supply[attr2]==v2)].name.values
    else:
        neighbors = Supply[(Supply[attr1]==v1) | (Supply[attr2]==v2)].name.values
    return neighbors

In [88]:
Demand['neighborSupply'] = Demand[['target1', 'target2', 'v1', 'v2']].apply(lambda x:findDemandNeighbours(*x),axis=1)


In [89]:
Demand.head()

,target1,target2,demand,penalty,V,name,v1,v2,S,theta,ksi,delivered,alpha,neighborSupply
187,4,1,660000.0,10,2.0,d187,7,69,787742.0,0.8378,0.85,669582.0,0,"[s13807, s13817, s13827, s13837, s13847, s1385..."
135,4,1,660000.0,10,2.0,d135,3,91,795219.0,0.8300,0.82,652079.0,0,"[s18203, s18213, s18223, s18233, s18243, s1825..."
717,4,1,620000.0,10,2.0,d717,5,71,784486.0,0.7903,0.78,611899.0,0,"[s14205, s14215, s14225, s14235, s14245, s1425..."
271,4,1,620000.0,10,2.0,d271,5,18,799621.0,0.7754,0.78,623704.0,0,"[s3605, s3615, s3625, s3635, s3645, s3655, s36..."
590,4,1,620000.0,10,3.0,d590,7,99,806589.0,0.7687,0.78,629139.0,0,"[s19807, s19817, s19827, s19837, s19847, s1985..."


In [90]:
DemandNeighbors = pd.DataFrame()
DemandNeighbors['Supply'] = Demand['neighborSupply']
DemandNeighbors['Demand'] = Demand['name']

In [91]:
tp =  DemandNeighbors.explode("Supply").groupby(["Supply"])['Demand'].apply(list)


In [92]:
SupplyNeighbors = pd.DataFrame()
SupplyNeighbors['name'] = tp.index
SupplyNeighbors['neighborDemand'] = tp.values

In [93]:
Supply = Supply.merge(SupplyNeighbors,on='name',how='left')

In [94]:
Supply.neighborDemand = Supply.neighborDemand.apply(lambda x: x if isinstance(x, list) else [])

In [95]:
# Supply[['impression','name','neighborDemand']].to_csv("Supply.csv",index=False)

In [96]:
# Demand[['demand', 'penalty', 'V', 'name', 'S',
#        'theta','neighborSupply']].to_csv("Demand.csv",index=False)

In [97]:
Demand.columns

Index(['target1', 'target2', 'demand', 'penalty', 'V', 'name', 'v1', 'v2', 'S',
       'theta', 'ksi', 'delivered', 'alpha', 'neighborSupply'],
      dtype='object')

In [98]:
Supply.head()

,gender,location,age,device,genre,impression,name,beta,neighborDemand
0,0,0,0,0,0,22702.0,s0,0,[]
1,0,0,0,0,1,20152.0,s1,0,"[d18, d132, d148, d164, d268, d326, d370, d438..."
2,0,0,0,0,2,19988.0,s2,0,"[d86, d148, d241, d306, d326, d392, d423, d463..."
3,0,0,0,0,3,21125.0,s3,0,"[d29, d86, d212, d264, d362, d418, d444, d463,..."
4,0,0,0,0,4,20814.0,s4,0,"[d16, d45, d167, d306, d415, d418, d444, d526,..."


In [99]:
Demand.head()

,target1,target2,demand,penalty,V,name,v1,v2,S,theta,ksi,delivered,alpha,neighborSupply
187,4,1,660000.0,10,2.0,d187,7,69,787742.0,0.8378,0.85,669582.0,0,"[s13807, s13817, s13827, s13837, s13847, s1385..."
135,4,1,660000.0,10,2.0,d135,3,91,795219.0,0.8300,0.82,652079.0,0,"[s18203, s18213, s18223, s18233, s18243, s1825..."
717,4,1,620000.0,10,2.0,d717,5,71,784486.0,0.7903,0.78,611899.0,0,"[s14205, s14215, s14225, s14235, s14245, s1425..."
271,4,1,620000.0,10,2.0,d271,5,18,799621.0,0.7754,0.78,623704.0,0,"[s3605, s3615, s3625, s3635, s3645, s3655, s36..."
590,4,1,620000.0,10,3.0,d590,7,99,806589.0,0.7687,0.78,629139.0,0,"[s19807, s19817, s19827, s19837, s19847, s1985..."


In [100]:
def g(alpha,theta,V,beta):
    return theta*(1+(alpha-beta)/V)

def computeBeta(alpha,theta,V):
    beta = (sum(theta) + sum(theta*alpha/V) - 1)/sum(theta/V)
    return beta

def checkBelongInterval(lower,upper,beta):
    return lower <= beta <= upper

def computeG(g):
    return sum(g)

In [101]:
def findBeta(neighborDemand):
    df = Demand[Demand.name.isin(neighborDemand)]
    upper_bound_max = sum(g(df.alpha,df.theta,df.V,0))
    
    
    if upper_bound_max <= 1: return 0
#     print(upper_bound_max)
    df['bound'] = df['alpha']+df['V']
    df = df.sort_values('bound')
    left,right = 1,df.shape[0]-1
    bounds = df.bound.values
   
    while (right >= left):
        
        median = right - (right-left)//2
        tp = df[median:]
#         tp2 = df[median-1:]
#         lower = bounds[median-1]
        upper = bounds[median]
        
        minNow = computeG(g(tp.alpha,tp.theta,tp.V,upper))
#         maxNow = computeG(g(tp2.alpha,tp2.theta,tp2.V,lower))
#         print("left,right,median,lower,upper,minNow,MaxNow",left,right,median,lower,upper,minNow,maxNow)
        if minNow <= 1:
            
            right = median - 1
        else:
            left = median + 1
    if left>=df.shape[0]:
        return 0
    lower = bounds[left-1]
    upper = bounds[left]
    tp = df[left:]
    beta = computeBeta(tp.alpha,tp.theta,tp.V)

    if checkBelongInterval(lower,upper,beta):
#         print("lower,upper,beta=",round(lower,4),round(upper,4),round(beta,4))
#         print("find solution ",computeG(g(tp.alpha,tp.theta,tp.V,beta)))
#         print(g(tp.alpha,tp.theta,tp.V,upper))
        return beta
    return 0

In [102]:
%%time

# Supply['beta'] = Supply['neighborDemand'].apply(findBeta)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [103]:
# def postCheckBeta(beta,neighborDemand):
    
#     tp = Demand[Demand.name.isin(neighborDemand)][['alpha','theta','V']]
#     tp['bound'] = tp['alpha'] + tp['V']
#     tp = tp.sort_values('bound')
#     tp2 = tp[tp.bound>=beta]
# #     a = ((tp.alpha-beta)/tp.V +1)*tp.theta
#     a =g(tp2.alpha,tp2.theta,tp2.V,beta)
#     a2 = g(tp.alpha,tp.theta,tp.V,beta)
#     value = round(sum(a))
#     if value!=1:
#         print(value,1)
#         print(a2)
#     return value!=1

In [104]:
def computeAlpha(theta,V,beta,s,d):
    return (d + sum(s*theta*beta/V) - sum(s*theta))/sum(s*theta/V)
def findAlpha(demand, penalty, V,S,theta,neighborSupply):
    df = Supply[Supply.name.isin(neighborSupply)]
    df['bound'] = df.beta - V
    df['bound'] = df['bound'].apply(lambda x: 0 if x<0 else x)
    df = df[df.bound<=penalty].sort_values("bound")
    upper_bound_max = sum(g(penalty,theta,V,df.beta)*df.impression)
    
    if upper_bound_max < demand:return penalty
    

    left,right = 1,df.shape[0]-2
    bounds = df.bound.values
    median = right - (right-left)//2
    while (right >= left):
        median = right - (right-left)//2
        tp = df[:median+1]
        lower = bounds[median]
        minNow = computeG(g(lower,theta,V,tp.beta)*tp.impression)
        if minNow <= demand:

            left = median + 1
        else:
            right = median - 1
    if left<=0:
        return penalty
    lower = bounds[left-1]
    upper = bounds[left]
    tp = df[:left]
    alpha = computeAlpha(theta,V,tp.beta,tp.impression,demand)
    
    if checkBelongInterval(lower,upper,alpha):
        
        return alpha
    return penalty

In [105]:
%%time
# Demand['alpha'] = 0
# Demand['alpha'] = Demand[['demand', 'penalty', 'V', 'S','theta','neighborSupply']].apply(lambda x:findAlpha(*x),axis=1)
       

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs


In [106]:

def postCheckAlpha(alpha,demand, penalty, V,S,theta,neighborSupply):
    tp = Supply[Supply.name.isin(neighborSupply)][['impression','beta']]
    tp['bound'] = tp['beta'] - V
    tp = tp[tp.bound<=penalty].sort_values('bound')
    a = tp.impression*theta*(1+(alpha-tp.beta)/V)
    value = round(sum(a[a>0]))
    if value!=demand:
        print(value,demand)
        print(a)
    return value!=demand
    

In [107]:
# sum(Demand[Demand.alpha!=Demand.penalty][['alpha','demand', 'penalty', 'V', 'S','theta','neighborSupply'
#                                      ]].apply(lambda x:postCheckAlpha(*x),axis=1))

In [108]:
%%time
iteration = 5
for i in range(iteration):
    print('iteration ',i)
    Supply['beta'] = Supply['neighborDemand'].apply(findBeta)
    Demand['alpha'] = Demand[['demand', 'penalty', 'V', 'S','theta','neighborSupply']].apply(lambda x:findAlpha(*x),axis=1)
#     print("check supply :",sum(Supply[Supply.beta>0][['beta',"neighborDemand"]].apply(lambda x:postCheckBeta(*x),axis=1)))
#     print("check demand :",sum(Demand[Demand.alpha!=Demand.penalty][['alpha','demand', 'penalty', 'V', 'S','theta','neighborSupply'
#                                      ]].apply(lambda x:postCheckAlpha(*x),axis=1)))

iteration  0


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

iteration  1
iteration  2
iteration  3
iteration  4
CPU times: user 18min 43s, sys: 3.29 s, total: 18min 46s
Wall time: 18min 46s


In [109]:
%%time
Supply['shat'] = Supply.impression
Supply['beta'] = Supply['neighborDemand'].apply(findBeta)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


CPU times: user 3min 40s, sys: 951 ms, total: 3min 41s
Wall time: 3min 41s


In [110]:
def computeBreakingPoints(theta,V,beta,alpha,s,shat):
    x = (shat/s/theta -1)*V + beta
    x[x>=alpha] = alpha
    x[x<beta-V] = beta-V
    x = set(x)
    x.add(0)
    x.add(alpha)
    
    return sorted(list(x))

# let F = sum(min(shat,s*g(alpha-beta)))
def computeF(shat,s,alpha,beta):
    part1 = s*np.maximum(0,theta*(1+(alpha-beta)/V))
    part2 = np.minimum(shat,part1)
#     print(part1,part2)
    return sum(part2)

def findIntervals(shat,s,beta,breakPoints):
    left,right = 0,len(breakPoints)-1
    while( right>=left ):
        median = right - (right-left)//2
        
        F = computeF(shat,s,breakPoints[median],beta)
        if F<=demand:
            left = median + 1
        else:
            right = median - 1
    if left == len(breakPoints):
        return breakPoints[-1],breakPoints[-1]
    lower = breakPoints[left-1]
    upper = breakPoints[left]
    return lower,upper

def findKsi(shat,s,beta,bins):
   
    left,right = 0,len(bins)-1
    while(right>=left):
        median = right - (right-left)//2
        F = computeF(shat,s,bins[median],beta)
        if F<=demand:
            left = median + 1
        else:
            right = median - 1
    if median == 0:
        return np.inf
    return bins[median]
    

In [111]:
%%time
step = 0.001
Ksi = []
Delivered = []
for items in Demand[['demand', 'penalty', 'V', 'S','theta',  'alpha', 'neighborSupply']].values:
    demand,penalty,V,S,theta,alpha,neighborSupply = items
    
    df = Supply[Supply.name.isin(neighborSupply)].copy()
    df['bound'] = df.beta - V
    df['bound'] = df['bound'].apply(lambda x: 0 if x<0 else x)
    df = df[df.bound<=penalty].sort_values("bound")
    breakPoints = computeBreakingPoints(theta,V,df.beta,alpha,df.impression,df.shat)
    maxF = computeF(df.shat,df.impression,alpha,df.beta)
    minF = computeF(df.shat,df.impression,0,df.beta)
    if minF<=demand<=maxF:
        intervals = findIntervals(df.shat,df.impression,df.beta,breakPoints)
        
        num = int((intervals[1]-intervals[0])/step)
        bins = np.linspace(intervals[0],intervals[1],num)
        if len(bins)==0:
            ksi = intervals[0]
        else:
            ksi = findKsi(df.shat,df.impression,df.beta,bins)
        delivered = np.minimum(df.shat,df.impression*np.maximum(0,theta*(1+(ksi-df.beta)/V)))
        Supply.loc[Supply.index.isin(delivered.index),'shat'] -= delivered
    
        Ksi.append(ksi)
        Delivered.append(sum(delivered))
    else:
        ksi = np.inf
        delivered = np.minimum(df.shat,df.impression*np.maximum(0,theta*(1+(ksi-df.beta)/V)))
        Supply.loc[Supply.index.isin(delivered.index),'shat'] -= delivered
        Ksi.append(ksi)
        Delivered.append(sum(delivered))

CPU times: user 39.7 s, sys: 250 ms, total: 40 s
Wall time: 40.1 s


In [112]:
Demand['Ksi'] = Ksi
Demand['Delivered'] = Delivered

In [113]:
Demand['Undelivered'] = Demand.demand - Demand.Delivered
Demand['Undelivered'] = Demand['Undelivered'].apply(lambda x: 0 if x<0 else x)
Demand['undelivered'] = Demand.demand - Demand.delivered
Demand['undelivered'] = Demand['undelivered'].apply(lambda x: 0 if x<0 else x)

In [114]:
def evaluateFirstPart(demand,penalty,V,theta,ksi,neighborSupply):

    
    df = Supply[Supply.name.isin(neighborSupply)]
#     print(df.head())
    x = np.maximum(0,theta*(1+(ksi-df.beta)/V))
#     print(x)
    part1 = (x-theta)**2*df.impression*V/theta
    
    return sum(part1)

# Demand['object2'] = Demand[['demand','penalty','V','theta','Ksi','neighborSupply']].apply(lambda x:evaluateFirstPart(*x),axis=1)
# Demand.head(5)[['demand','penalty','V','theta','alpha','neighborSupply']].apply(lambda x:evaluateFirstPart(*x),axis=1)



In [115]:
(Demand.penalty*Demand.undelivered).sum()

862121910.0

In [116]:
(Demand.penalty*Demand.Undelivered).sum()-(Demand.penalty*Demand.undelivered).sum()

-307857520.28997004

In [117]:
Demand['object2'] = Demand[['demand','penalty','V','theta','alpha','neighborSupply']].apply(lambda x:evaluateFirstPart(*x),axis=1)

Demand['object2'] = Demand['object2'] + (Demand.penalty*Demand.Undelivered).sum()

In [118]:
def evaluateFirstPart2(demand,penalty,V,theta,ksi,neighborSupply):

    
    df = Supply[Supply.name.isin(neighborSupply)]
#     print(df.head())
#     x = np.maximum(0,theta*(1+(ksi-df.beta)/V))
#     print(x)
    if ksi>1:
        ksi = 1
    part1 = (ksi-theta)**2*df.impression*V/theta
    
    return sum(part1)


Demand['object1'] =  Demand[['demand','penalty','V','theta','ksi','neighborSupply']].apply(lambda x:evaluateFirstPart2(*x),axis=1)
Demand['object1'] = Demand['object1'] + (Demand.penalty*Demand.undelivered).sum()


In [119]:
Demand['object1'].sum()-Demand['object2'].sum()

312325572671.01416

In [120]:
Demand.undelivered.sum()-Demand.Undelivered.sum()

30785752.028997004

In [121]:
Demand.Undelivered.sum()

55426438.971002996

In [122]:
Demand.undelivered.sum()

86212191.0

In [123]:
Demand[['penalty','alpha','theta','S','demand','Undelivered','undelivered']]

,penalty,alpha,theta,S,demand,Undelivered,undelivered
187,10,10.000000,0.8378,787742.0,660000.0,251.943888,0.0
135,10,10.000000,0.8300,795219.0,660000.0,0.000000,7921.0
717,10,10.000000,0.7903,784486.0,620000.0,134.962476,8101.0
271,10,10.000000,0.7754,799621.0,620000.0,204.558342,0.0
590,10,10.000000,0.7687,806589.0,620000.0,0.000000,0.0
256,10,10.000000,0.7562,793430.0,600000.0,386.722090,4928.0
557,10,10.000000,0.7539,795877.0,600000.0,33.599890,3089.0
705,10,10.000000,0.7538,795959.0,600000.0,0.000000,3030.0
163,10,10.000000,0.7528,797029.0,600000.0,0.000000,2225.0
183,10,10.000000,0.7509,799012.0,600000.0,0.000000,741.0


# Gradient Descent

In [151]:
def computeXij(alpha,theta,V,beta):
    
   
    return np.maximum(0,g(alpha,theta,V,beta))
def computeGradientsAlpha(alpha,theta,V,demand,s,neighborSupply):
    
    df = Supply[Supply.name.isin(neighborSupply)]
    xij = computeXij(alpha,theta,V,df.beta)
    return demand - sum(s*xij)


def computeGradientsBeta(beta,s,neighborDemand):
    df = Demand[Demand.name.isin(neighborDemand)]
    xij = computeXij(df.alpha,df.theta,df.V,beta)
    return s*sum(xij) - s



In [152]:
Supply[Supply.beta==0].shat.sum()

972872.2020930812

In [68]:
Supply[Supply.beta>0].shat.sum()

0.0

In [75]:
Supply.impression.sum()-Demand.Delivered.sum()

637913.721075356

In [79]:
Demand[Demand.Undelivered>=270000.000000]

,target1,target2,demand,penalty,V,name,v1,v2,S,theta,ksi,delivered,alpha,neighborSupply,alpha2,demand2,S2,Ksi,Delivered,Undelivered,undelivered
872,4,1,300000.0,1.709,3.0,d872,1,47,401230.0,0.7477,inf,72225.0,1.709000,"[s9401, s9411, s9421, s9431, s9441, s9451, s94...",0.323798,3.0,4.01230,inf,0.000000,300000.000000,227775.0
148,4,1,280000.0,0.386,3.0,d148,3,66,399864.0,0.7002,inf,87969.0,0.386000,"[s13203, s13213, s13223, s13233, s13243, s1325...",0.386000,2.8,3.99864,inf,0.000000,280000.000000,192031.0
885,3,1,290000.0,0.888,3.0,d885,3,23,999635.0,0.2901,0.31,309886.0,0.888000,"[s4630, s4631, s4632, s4633, s4634, s4635, s46...",0.888000,2.9,9.99635,inf,0.000000,290000.000000,0.0
809,3,1,290000.0,1.114,2.0,d809,2,74,1000855.0,0.2898,0.27,270229.0,1.114000,"[s14820, s14821, s14822, s14823, s14824, s1482...",0.381281,2.9,10.00855,inf,0.000000,290000.000000,19771.0
262,1,3,270000.0,1.782,2.0,d262,73,3,1000087.0,0.2700,0.27,256723.0,1.782000,"[s14630, s14631, s14632, s14633, s14634, s1463...",0.519281,2.7,10.00087,inf,0.000000,270000.000000,13277.0
326,1,3,270000.0,1.442,2.0,d326,84,1,1000184.0,0.2700,0.27,270048.0,1.442000,"[s16810, s16811, s16812, s16813, s16814, s1681...",0.255379,2.7,10.00184,inf,0.000000,270000.000000,0.0
117,3,1,270000.0,0.128,2.0,d117,2,60,999973.0,0.2700,0.31,278942.0,0.128000,"[s12020, s12021, s12022, s12023, s12024, s1202...",0.128000,2.7,9.99973,inf,0.000000,270000.000000,0.0
901,1,3,270000.0,0.649,1.0,d901,5,2,1000636.0,0.2698,0.40,281025.0,0.649000,"[s1020, s1021, s1022, s1023, s1024, s1025, s10...",0.359522,2.7,10.00636,inf,0.000000,270000.000000,0.0
126,4,2,300000.0,0.291,3.0,d126,1,4,7999297.0,0.0375,0.05,386352.0,0.177564,"[s161, s171, s181, s191, s361, s371, s381, s39...",0.291000,3.0,79.99297,inf,0.000000,300000.000000,0.0
365,4,2,290000.0,0.526,4.0,d365,6,4,7999242.0,0.0363,0.05,389417.0,0.526000,"[s166, s176, s186, s196, s366, s376, s386, s39...",0.526000,2.9,79.99242,inf,0.000000,290000.000000,0.0


In [82]:
Demand.undelivered.sum()-Demand.Undelivered.sum()

-116870515.68232024

In [81]:
Demand.Undelivered.sum()

156050907.68232024

In [51]:
Demand.demand.max(),Supply.impression.max()

(390000.0, 10405.0)

In [93]:
Demand.delivered.sum()

336238051.0

In [94]:
Supply.impression.sum()

400011174.0

In [97]:
SHat.impression.sum()+Demand.delivered.sum()

400011174.0

In [59]:
scale = 100000
Demand['alpha2'] = np.random.uniform(0,1,numDemand).round(3)
Supply["beta2"] = np.random.uniform(0,1,numSupply).round(3)
Demand['demand2'] = Demand.demand/scale
Demand['S2'] = Demand.S/scale
Supply['impression2'] = Supply.impression/scale
lr = 0.0001

In [60]:
%%time
for iteration in range(10):
    print("iter ",iteration)
    GradientsAlpha = Demand[['alpha2','theta','V','demand2','S2','neighborSupply']].apply(lambda x:computeGradientsAlpha(*x),axis=1)
    GradientsBeta = Supply[['beta2','impression2','neighborDemand']].apply(lambda x:computeGradientsBeta(*x),axis=1)
    Demand['alpha2'] -= lr*GradientsAlpha
    Supply['beta2'] -= lr*GradientsBeta
    Demand['alpha2'] = Demand[['alpha2','penalty']].apply(lambda x: x[0] if 0<=x[0]<=x[1] else x[1],axis=1)
    Supply['beta2'] = Supply['beta2'].apply(lambda x: x if x>0 else 0)


iter  0
iter  1
iter  2
iter  3
iter  4
iter  5
iter  6
iter  7
iter  8
iter  9
CPU times: user 8min 2s, sys: 869 ms, total: 8min 3s
Wall time: 8min 2s


In [58]:
Demand[Demand.alpha2!=Demand.penalty].shape

(31, 17)

In [55]:
Demand.alpha2

554    1.145
991    1.399
979    1.632
20     1.475
866    1.091
692    0.594
368    0.626
884    1.903
928    1.998
139    1.432
260    1.264
342    1.288
292    0.857
776    1.618
704    1.081
741    0.294
872    1.709
987    0.178
932    0.185
826    1.029
278    1.495
148    0.386
629    0.247
921    0.766
119    1.342
116    0.617
131    0.193
893    1.891
490    0.511
954    1.298
775    0.661
480    0.723
142    0.919
963    1.573
846    0.184
703    1.299
26     0.309
198    0.947
199    1.294
355    1.489
448    0.184
999    0.762
810    0.665
74     1.237
400    0.066
757    0.239
888    1.263
844    0.252
789    0.273
578    1.082
340    1.319
821    1.198
203    0.648
716    0.341
104    0.731
793    1.708
202    0.129
55     1.078
506    1.236
624    0.907
913    1.282
764    0.548
529    1.727
475    0.446
990    0.541
40     1.585
250    1.165
378    1.491
218    0.519
982    0.428
130    0.305
706    0.034
718    0.609
494    0.904
169    0.113
572    1.444
900    1.598

In [318]:
Demand.alpha

554    1.145000
991    1.399000
979    1.632000
20     1.475000
866    1.091000
692    0.594000
368    0.626000
884    1.903000
139    1.432000
928    1.998000
260    1.264000
342    1.288000
292    0.857000
776    1.618000
704    1.081000
741    0.294000
872    1.709000
987    0.178000
932    0.185000
826    1.029000
278    1.495000
148    0.386000
629    0.247000
921    0.766000
119    1.342000
116    0.617000
131    0.193000
893    1.891000
490    0.511000
775    0.661000
954    1.298000
480    0.723000
142    0.919000
963    1.573000
846    0.184000
703    1.299000
26     0.309000
198    0.947000
199    1.294000
355    1.489000
448    0.184000
999    0.762000
810    0.665000
74     1.237000
400    0.066000
888    1.263000
757    0.239000
844    0.252000
789    0.273000
578    1.082000
340    1.319000
821    1.198000
203    0.648000
716    0.341000
104    0.731000
793    1.708000
202    0.129000
55     1.078000
506    1.236000
624    0.907000
913    1.282000
764    0.548000
529    1

In [323]:
Supply[['beta','beta2']]

,beta,beta2
0,0.000000,1.008133
1,0.000000,0.179685
2,0.000000,0.011769
3,0.000000,0.576191
4,0.000000,0.357428
...,...,...
39995,1.728170,0.068965
39996,1.783496,0.213907
39997,2.106414,0.733014
39998,1.982634,0.376068


# OR Tools

In [ ]:
solver = pywraplp.Solver('SolveSimpleSystem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

In [ ]:
Demand = Demand.sample(100).reset_index(drop=True)
Supply = Supply.sample(1000).reset_index(drop=True)

In [ ]:
Demand['ind'] = Demand.index
Supply['ind'] = Supply.index

In [ ]:
Demand.head()

In [ ]:
Supply.head()

In [ ]:
import numpy as np
from scipy.optimize import minimize

def objective(x):
    return x[0]*x[3]*(x[0]+x[1]+x[2])+x[2]

def constraint1(x):
    return x[0]*x[1]*x[2]*x[3]-25.0

def constraint2(x):
    sum_eq = 40.0
    for i in range(4):
        sum_eq = sum_eq - x[i]**2
    return sum_eq

# initial guesses
n = 4
x0 = np.zeros(n)
x0[0] = 1.0
x0[1] = 5.0
x0[2] = 5.0
x0[3] = 1.0

# show initial objective
print('Initial SSE Objective: ' + str(objective(x0)))

# optimize
b = (1.0,5.0)
bnds = (b, b, b, b)
con1 = {'type': 'ineq', 'fun': constraint1} 
con2 = {'type': 'eq', 'fun': constraint2}
cons = ([con1,con2])
solution = minimize(objective,x0,method='SLSQP',\
                    bounds=bnds,constraints=cons)
x = solution.x

# show final objective
print('Final SSE Objective: ' + str(objective(x)))

# print solution
print('Solution')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))
print('x3 = ' + str(x[2]))
print('x4 = ' + str(x[3]))


In [ ]:
x = np.random.uniform(0,1,100*1000)
x = np.array(x).reshape(1000,100).T

In [ ]:
equations = []

for items in Demand[['target1', 'target2', 'demand', 'v1', 'v2','theta','V',"penalty","ind"]].values:
    
    target1,target2,dj,v1,v2,theta,V,penalty,ind = items
    attr1 = attrNames[int(target1)]
    attr2 = attrNames[int(target2)]
  
    if attr1 != attr2:
        s = Supply[(Supply[attr1]==v1)&(Supply[attr2]==v2)]
       
    else:
        s = Supply[(Supply[attr1]==v1) | (Supply[attr2]==v2)]
 

    

    xij = x[int(ind)][s.index]
    
    uj = dj - sum(s.impression*xij)
    
    equations.append(s.impression*V*(xij-theta)*(xij-theta) + uj*penalty)
    constrains1 = xij>=0

In [ ]:
def constraint1(x):
    return x[0]*x[1]*x[2]*x[3]-25.0


In [ ]:
b = (0,1)
bnds = tuple([b]*100*1000)
solution = minimize(sum(equations),x,method='SLSQP',\
                    bounds=bnds,constraints=constrains1)

In [ ]:
tuple([(0,1)]*10)

In [ ]:
x = []

for j in range(Demand.shape[0]):
    for i in range(Supply.shape[0]):
        x.append(solver.NumVar(0, 1, 'x_{0}_{1}'.format(j,i)))

In [ ]:
x = np.array(x).reshape(Supply.shape[0],Demand.shape[0]).T

In [ ]:
equations = []

for items in Demand[['target1', 'target2', 'demand', 'v1', 'v2','theta','V',"penalty","ind"]].values:
    
    target1,target2,dj,v1,v2,theta,V,penalty,ind = items
    attr1 = attrNames[int(target1)]
    attr2 = attrNames[int(target2)]
  
    if attr1 != attr2:
        s = Supply[(Supply[attr1]==v1)&(Supply[attr2]==v2)]
       
    else:
        s = Supply[(Supply[attr1]==v1) | (Supply[attr2]==v2)]
 
    xij = x[int(ind)][s.index]

    uj = dj - sum(s.impression*xij)
    equations.append(s.impression*V*(xij-theta) + uj*penalty)
    
#     a = sum(s.impression*xij) + uj>=dj
#     solver.Add(uj>=np.array([0*s.shape[0]]))
    
#     for si in s:
#         si*V*
#     
#     shati = SHat[(SHat[attr1]==v1) | (SHat[attr2]==v2)].impression.values

#     ksi = findKsi(shati,si,dj,step)

#     delivered = np.minimum(shati,si*ksi).round()

#     SHat.loc[(SHat[attr1]==v1) | (SHat[attr2]==v2),"impression"] -= delivered
#     Q.append(ksi)
#     Delivered.append(sum(delivered))

In [ ]:
np.array([0]*10)

In [ ]:
solver.MakeProd(x[0][1],x[1][1])

In [ ]:
solver.

In [ ]:
import gurobipy as gp

In [ ]:
import sys

In [ ]:
import sys
sys.path.append('/Users/hzhu/miniconda3/lib/python3.7/site-packages/gurobipy/')

In [ ]:
import gurobipy